In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix,accuracy_score
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/dataset/Preprocessed_Tweet_DF_new_3_1_2023.csv")

In [5]:
df

,account,ioc_value,text,isValid,preprocess_text
0,aerezona,https://voonze.com/how-to-avoid-ransomware-att...,How to avoid ransomware attacks #attacks #avoi...,0,avoid ransomware attack attack avoid ransomware
1,HeliosCert,87dcceb0f3215a3e7917874aa32d2738e575a400ed1a74...,Sample submitted\n2021-05-04 02:45:01\nDionaea...,1,sample submit dionaea honeypot protocol smbd s...
2,nixcraft,https://google.github.io/security-research/poc...,CVE-2021-22555 is a 15 years old heap out-of-b...,1,cve year old heap outofbounds write vulnerabil...
3,atlasinfosys,https://stuf.in/b6mpi0,The global average cost of a #cyberattack is $...,0,global average cost cyberattack million yikes ...
4,romonlyht,http://specialnotice.cnewamazoncardservice.cyou/,https://t.co/CTs1mC6BpX\n\nNo.3 (NXDOMAIN)\n\n...,1,nxdomain receive [ip] [defanged] phishing amazon
...,...,...,...,...,...
32583,ecarlesi,http://report-id-9458.xyz/,Threat on hxxp://report-id-9458[.]xyz/ #phishi...,1,threat [defanged] phishing facebook
32584,CyberIgor,https://lnkd.in/dsMZD7X,@KyleHanslovan appears on CNBC to inject some ...,0,appear cnbc inject muchneeded sanity hypedrive...
32585,dtnewsonline,https://digitalterminal.in/news/sophos-detecte...,Sophos Detected Dangerous Malware Targeting Di...,0,sophos detect dangerous malware target discord...
32586,cyberarms_,https://t.co/oFawA363Xh,How to stay safe from location tracking app?\n...,0,stay safe location track app cyberarmsae cyber...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32588 entries, 0 to 32587
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   account          32588 non-null  object
 1   ioc_value        32588 non-null  object
 2   text             32588 non-null  object
 3   isValid          32588 non-null  int64 
 4   preprocess_text  32578 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.2+ MB


In [5]:
df['isValid'].value_counts()

0    20188
1    12400
Name: isValid, dtype: int64

In [6]:
df['preprocess_text'] = df['preprocess_text'].fillna('[NULL]')

In [ ]:
#@title
def create_wordcloud(words):
    wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(words)
    plt.figure(figsize=(10, 7))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()

In [ ]:
#@title
subset=df[df.isValid==0]
text=subset.preprocess_text.values
words =" ".join(text)
create_wordcloud(words)

In [ ]:
#@title
subset=df[df.isValid==1]
text=subset.preprocess_text.values
words =" ".join(text)
create_wordcloud(words)

In [ ]:
#@title
def get_corpus(text):
    words = []
    for i in text:
        for j in i.split():
            words.append(j.strip())
    return words
corpus = get_corpus(df.preprocess_text)
corpus[:5]

In [ ]:
#@title
from collections import Counter
counter = Counter(corpus)
most_common = counter.most_common(10)
most_common = dict(most_common)
most_common

In [ ]:
#@title
from sklearn.feature_extraction.text import CountVectorizer
def get_top_text_ngrams(corpus, n, g):
    vec = CountVectorizer(ngram_range=(g, g)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
#@title
plt.figure(figsize = (16,9))
most_common_uni = get_top_text_ngrams(df.preprocess_text,10,1)
most_common_uni = dict(most_common_uni)
sns.barplot(x=list(most_common_uni.values()),y=list(most_common_uni.keys()))

In [ ]:
#@title
plt.figure(figsize = (16,9))
most_common_bi = get_top_text_ngrams(df.preprocess_text,10,2)
most_common_bi = dict(most_common_bi)
sns.barplot(x=list(most_common_bi.values()),y=list(most_common_bi.keys()))

In [ ]:
#@title
plt.figure(figsize = (16,9))
most_common_tri = get_top_text_ngrams(df.preprocess_text,10,3)
most_common_tri = dict(most_common_tri)
sns.barplot(x=list(most_common_tri.values()),y=list(most_common_tri.keys()))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df['preprocess_text'], 
                                                    df['isValid'], 
                                                    test_size=0.2, 
                                                    random_state=8)

In [8]:
X=df['preprocess_text']

In [9]:
y=df['isValid']

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud ,STOPWORDS
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize 

In [13]:
ngram_range = (1,2)
min_df = 5
max_df = 1.
max_features = 300

In [14]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.28890545 ... 0.269635   0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(6518, 300)


In [15]:
from sklearn.ensemble import RandomForestClassifier
model  = RandomForestClassifier(max_depth=30, n_estimators=300, random_state=1)
model.fit(features_train, labels_train)
model_predictions = model.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, model_predictions))
print(classification_report(labels_test, model_predictions))

Accuracy:  0.9799018103712795
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4040
           1       0.99      0.96      0.97      2478

    accuracy                           0.98      6518
   macro avg       0.98      0.98      0.98      6518
weighted avg       0.98      0.98      0.98      6518



In [ ]:
with open('rf_classifier', 'wb') as picklefile:
    pickle.dump(model,picklefile)

In [ ]:
with open('tfidf', 'wb') as picklefile:
    pickle.dump(tfidf,picklefile)

In [ ]:
from sklearn.model_selection import GridSearchCV
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(model, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(features_train, labels_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


In [ ]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
X = tfidf.fit_transform(X).toarray()
y = y


In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
model =  DecisionTreeClassifier(random_state=1)

In [ ]:
scoring = ['accuracy','precision', 'recall', 'f1']
score=cross_val_score(model,X,y,scoring='accuracy',cv=10)

In [ ]:
score

array([0.973466  , 0.97401879, 0.973466  , 0.97457159, 0.9795354 ,
       0.97511062, 0.98064159, 0.97566372, 0.97123894, 0.97787611])

In [ ]:
print(score.mean()*100)

97.55588759741117


In [11]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2023-01-03 07:13:33--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-01-03 07:13:33--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  4.78MB/s    in 2m 39s  

2023-01-03 07:16:12 (5.17 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [12]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [13]:
EMBEDDING_FILE = 'glove.6B.300d.txt'

https://www.kaggle.com/code/madz2000/nlp-using-glove-embeddings-99-87-accuracy/notebook

https://www.kaggle.com/code/hamishdickson/training-and-plotting-word2vec-with-bigrams

https://www.kaggle.com/code/jagannathrk/word2vec-cnn-text-classification

https://www.h2kinfosys.com/blog/word-embeddings-with-word2vec-tutorial-all-you-need-to-know/

https://nlp.stanford.edu/projects/glove/

In [14]:
# prepare tokenizer
token = Tokenizer()
token.fit_on_texts(df['preprocess_text'])
max_len= max(len(s) for s in df['preprocess_text'])
vocab_size = len(token.word_index) + 1
# integer encode the documents
text = token.texts_to_sequences(df['preprocess_text'])
print(text)

[[352, 1, 10, 10, 352, 1], [16, 20, 21, 15, 19, 22, 17, 18, 6, 13, 11118, 5, 9, 7], [129, 134, 678, 7819, 2326, 469, 57, 224, 3241, 1559, 895, 1025, 896, 7, 1262, 3242, 1660, 216, 748, 299, 224, 1660, 2545, 12, 1834, 7, 90], [150, 574, 242, 11, 102, 2428, 563, 69, 60, 39, 74, 1711, 62, 353, 558, 361, 8, 309, 2, 1], [1142, 30, 6, 3, 2, 34], [11119, 7820, 5283, 2056, 267, 263, 127, 231, 11120, 268, 258, 269], [46, 27, 49, 944, 2139, 428], [42, 7821, 92, 698, 9, 372, 2057, 39, 504, 3243, 544, 307, 647, 669, 1, 222, 10, 1262, 293], [16, 20, 21, 15, 19, 22, 17, 18, 6, 13, 11121, 5, 9, 7], [975, 30, 328, 42, 1712, 975, 4120, 1560, 41, 5284, 7822, 3754], [42, 634, 159, 5285, 9, 10, 505, 14, 7, 4, 47, 43, 11, 26, 370, 212, 24, 5, 142, 94, 1, 49, 65, 124], [515, 64, 38, 11122, 11123, 2704, 670, 38, 66, 11124, 1423, 63, 202, 240], [1026, 38, 11125, 1777, 354, 226, 391, 11126, 1976, 16, 110, 365, 470, 690, 117, 96, 227, 5], [229, 27, 4629, 160, 31, 177, 923, 7823, 342, 44, 588, 366, 880, 1661, 46

In [35]:
with open('/content/drive/MyDrive/Twitter_Work_Code/output/tokenizer_3_1_23', 'wb') as picklefile:
    pickle.dump(token,picklefile)

In [15]:
MAX_SEQUENCE_LENGTH = 128
max_len=128
MAX_NB_WORDS = 10000
EMBEDDING_DIM=300
VALIDATION_SPLIT = 0.1
traina = True #embeedding layer trainable

In [16]:
# pad documents to a max length of 4 words
text = pad_sequences(text, maxlen=max_len, padding='post')
print(text)

[[ 352    1   10 ...    0    0    0]
 [  16   20   21 ...    0    0    0]
 [ 129  134  678 ...    0    0    0]
 ...
 [1205  239  853 ...    0    0    0]
 [ 415  404 2018 ...    0    0    0]
 [  56   14  449 ...    0    0    0]]


In [17]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [18]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in token.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [19]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df['isValid'])
y = to_categorical(y)
y[:2]

array([[1., 0.],
       [0., 1.]], dtype=float32)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(np.array(text), y, test_size=0.2, stratify=y,random_state=100)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, stratify=y_test)

In [21]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D

In [22]:
keras_model = Sequential()
keras_model.add(Embedding(vocab_size,
                        EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        input_length=max_len,
                        trainable=False))
keras_model.add(Dropout(0.2))
keras_model.add(Conv1D(50, 3, activation='relu', padding='same', strides=1))
keras_model.add(Conv1D(50, 3, activation='relu', padding='same', strides=1))
keras_model.add(MaxPool1D())
keras_model.add(Dropout(0.2))
keras_model.add(Conv1D(100, 3, activation='relu', padding='same', strides=1))
keras_model.add(Conv1D(100, 3, activation='relu', padding='same', strides=1))
keras_model.add(MaxPool1D())
keras_model.add(Dropout(0.2))
keras_model.add(Conv1D(200, 3, activation='relu', padding='same', strides=1))
keras_model.add(Conv1D(200, 3, activation='relu', padding='same', strides=1))
keras_model.add(GlobalMaxPool1D())
keras_model.add(Dropout(0.2))
keras_model.add(Dense(200))
keras_model.add(Activation('relu'))
keras_model.add(Dropout(0.2))
keras_model.add(Dense(2))
keras_model.add(Activation('sigmoid'))
keras_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

In [23]:
keras_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 128, 300)          8515800   
                                                                 
 dropout (Dropout)           (None, 128, 300)          0         
                                                                 
 conv1d (Conv1D)             (None, 128, 50)           45050     
                                                                 
 conv1d_1 (Conv1D)           (None, 128, 50)           7550      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 64, 50)           0         
 )                                                               
                                                                 
 dropout_1 (Dropout)         (None, 64, 50)            0         
                                                        

In [24]:
keras_model.fit(x_train, y_train, batch_size=16, epochs=50,  validation_data=(x_val,y_val))

Epoch 1/50
1630/1630 [==============================] - 21s 7ms/step - loss: 0.1348 - accuracy: 0.9418 - val_loss: 0.0567 - val_accuracy: 0.9828
Epoch 2/50
1630/1630 [==============================] - 10s 6ms/step - loss: 0.0575 - accuracy: 0.9820 - val_loss: 0.0444 - val_accuracy: 0.9856
Epoch 3/50
1630/1630 [==============================] - 10s 6ms/step - loss: 0.0470 - accuracy: 0.9851 - val_loss: 0.0466 - val_accuracy: 0.9859
Epoch 4/50
1630/1630 [==============================] - 10s 6ms/step - loss: 0.0408 - accuracy: 0.9873 - val_loss: 0.0448 - val_accuracy: 0.9868
Epoch 5/50
1630/1630 [==============================] - 10s 6ms/step - loss: 0.0358 - accuracy: 0.9889 - val_loss: 0.0423 - val_accuracy: 0.9880
Epoch 6/50
1630/1630 [==============================] - 10s 6ms/step - loss: 0.0335 - accuracy: 0.9895 - val_loss: 0.0394 - val_accuracy: 0.9890
Epoch 7/50
1630/1630 [==============================] - 10s 6ms/step - loss: 0.0302 - accuracy: 0.9911 - val_loss: 0.0471 - val_ac

In [25]:
y_test = np.argmax(y_test, axis=1)
y_pred_enc = keras_model.predict(x_test)
y_pred_enc = np.argmax(y_pred_enc, axis=1)

102/102 [==============================] - 1s 5ms/step


In [44]:
y_pred_enc

array([0, 0, 0, ..., 0, 0, 0])

In [26]:
print(classification_report(y_test,y_pred_enc))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2019
           1       0.99      0.97      0.98      1240

    accuracy                           0.99      3259
   macro avg       0.99      0.98      0.99      3259
weighted avg       0.99      0.99      0.99      3259



In [27]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
print(f1_score(y_test,y_pred_enc,average='weighted'))

0.9870834027706936


In [28]:
print(confusion_matrix(y_test,y_pred_enc))

[[2012    7]
 [  35 1205]]


In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

actual = numpy.random.binomial(1,.9,size = 1000)
predicted = numpy.random.binomial(1,.9,size = 1000)

confusion_matrix = metrics.confusion_matrix(y_test, y_pred_enc)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show() 

In [37]:
with open('/content/drive/MyDrive/Twitter_Work_Code/output/text_classifier_3_1_23', 'wb') as picklefile:
    pickle.dump(keras_model,picklefile)

In [ ]:
with open('/content/drive/MyDrive/output/text_classifier_3_1_23', 'rb') as training_model:
    model = pickle.load(training_model)

In [ ]:
new_df = pd.read_csv("/content/drive/MyDrive/output/Final_Unique_Tweets.csv")

In [ ]:
new_df

,Unnamed: 0,Unnamed: 0.1,Author ID,User Name,Screen Name,Tweet ID,Created At,Text,Like Count,Quote Count,...,Conversation ID,Referenced Tweets,ID of Referenced Tweets,Type of Referenced Tweets,Attachments,Location tagged by the user,Language,Sensitive URL,Reply Settings,Source
0,2414663,59047,4.501058e+08,ItsTrigger,Trigger,1.344795e+18,2021-01-01 00:00:02+00:00,2020 was rough for us all. I don't usually wri...,7.0,0.0,...,1.344795e+18,NaN,NaN,NaN,NaN,NaN,en,False,everyone,Twitter Web App
1,2414662,59046,2.918316e+08,Paula_Piccard,Paula Piccard,1.344796e+18,2021-01-01 00:00:26+00:00,SolarWinds hackers accessed Microsoft source c...,8.0,0.0,...,1.344796e+18,NaN,NaN,NaN,{'media_keys': ['3_1344795579990351872']},NaN,en,False,everyone,SmarterQueue
2,2414661,59045,5.693841e+08,velez1066,Jorge Velez,1.344796e+18,2021-01-01 00:01:01+00:00,The pattern of #ransomware impacting manufactu...,0.0,0.0,...,1.344796e+18,NaN,NaN,NaN,NaN,NaN,en,False,everyone,Trend Micro
3,2414654,59038,9.154360e+17,MoustachedTrex,Johnny,1.344796e+18,2021-01-01 00:03:25+00:00,https://t.co/L4NQPYJ8K7 A list of everything I...,1.0,0.0,...,1.344796e+18,NaN,NaN,NaN,NaN,NaN,en,False,everyone,Twitter Web App
4,2414650,59034,1.059414e+09,An0n_Truth,Anonymous,1.344797e+18,2021-01-01 00:06:06+00:00,The latest The An0nym0us Truth! https://t.co/X...,0.0,0.0,...,1.344797e+18,NaN,NaN,NaN,NaN,NaN,en,False,everyone,Paper.li
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430182,50,50,1.384262e+18,hankobaggins,hanko.sol,1.421619e+18,2021-07-31 23:49:53+00:00,@cryptoFLman @BradRSylvain @BoringProtocol FIN...,6.0,0.0,...,1.420470e+18,[<ReferencedTweet id=1420470148188737542 type=...,1.420470e+18,replied_to,NaN,NaN,en,False,everyone,Twitter Web App
430183,48,48,9.503332e+17,HeliosCert,HELIOS CERT,1.421619e+18,2021-07-31 23:50:02+00:00,Sample submitted\n2021-07-31 23:50:02\nDionaea...,1.0,0.0,...,1.421619e+18,NaN,NaN,NaN,NaN,NaN,en,False,everyone,Helios CDC Cert
430184,46,46,2.469117e+08,PMProuk,Jason Normanton 🤍💙💛,1.421619e+18,2021-07-31 23:50:21+00:00,"As cyberattacks skyrocket, Canada needs to wor...",0.0,0.0,...,1.421619e+18,NaN,NaN,NaN,NaN,NaN,en,False,everyone,TwinyBots
430185,38,38,9.202597e+17,cyberinform,Jose Lagdameo Jr,1.421620e+18,2021-07-31 23:52:04+00:00,#Ransomware attacks in #2021 have already surp...,0.0,0.0,...,1.421620e+18,NaN,NaN,NaN,{'media_keys': ['3_1421619699301892102']},NaN,en,False,everyone,Buffer


In [ ]:
model.predict(sample_dataset)

In [ ]:
yhat = model.predict(x_test)
classes_x=np.argmax(yhat,axis=1)
classes_x

57/57 [==============================] - 0s 4ms/step


array([0, 0, 1, ..., 0, 1, 1])